<a href="https://colab.research.google.com/github/juliawol/WB_Knowledge_Base/blob/main/Fine_tuning_with_triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd
import random

# Load the original datasets with positive and negative examples
train_data_path = '/content/train_data.csv'
train_data_df = pd.read_csv(train_data_path)

# Define the terminology and its definitions as additional training examples
terminology = [
    ("ТП", "торговая площадка. Платформа Wildberries."),
    ("ПВЗ", "пункт выдачи заказов. Место, куда покупатели приходят за посылками."),
    ("ШК офиса", "уникальный штрихкод, который даёт доступ к рабочему интерфейсу NPOS."),
    ("ID офиса", "номер пункта выдачи в системе Wildberries."),
    ("ID менеджера", "номер учетной записи в системе Wildberries."),
    ("ШК", "штрихкод. На упаковке каждого товара и на приходных коробках."),
    ("Стикер", "помогает узнать информацию о заказе, но не используется для поиска товара."),
    ("Баркод", "штрихкод производителя. Используется для сверки данных о товаре."),
    ("QR-код", "двумерный штрихкод с информацией, расшифровывается сканером."),
    ("Волна или волнорез", "стеллаж, где хранятся товары."),
    ("Приходная коробка", "упаковка, в которой заказы приходят в пункт выдачи."),
    ("Невостребованный товар", "товар, который покупатель не забрал из ПВЗ в течение 12 дней."),
    ("Невозвратный товар", "товар, который нельзя вернуть."),
    ("Возвратная коробка", "упаковка, в которой невостребованные товары отправляют обратно на склад."),
    ("Возвратная наклейка", "элемент упаковки со штрихкодом и номером коробки."),
    ("Сейф-пакет", "специальная упаковка для ювелирных изделий и гаджетов.")
]

# Prepare triplet examples
triplet_examples = []

# Add terminology as (anchor, positive, negative) triplets
for term, definition in terminology:
    # Define anchor and positive
    anchor = term
    positive = definition
    # Sample a negative definition from the terminology
    negative = random.choice([defn for t, defn in terminology if defn != positive])
    triplet_examples.append(InputExample(texts=[anchor, positive, negative]))

# Add original training pairs from `train_data_df` with predefined negatives
for _, row in train_data_df.dropna().iterrows():
    question = row['Question']
    positive_chunk = row['Chunk']
    negative_chunk = row['Hard negative']  # Use the provided negative chunk
    triplet_examples.append(InputExample(texts=[question, positive_chunk, negative_chunk]))

# Initialize SentenceTransformer model and DataLoader
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
train_dataloader = DataLoader(triplet_examples, shuffle=True, batch_size=16)

# Define triplet loss
train_loss = losses.TripletLoss(model=model, triplet_margin=1.0)

# Fine-tune the model using triplet loss
num_epochs = 3  # For our extremely small dataset
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=100,
    show_progress_bar=True
)

# Save and reload the fine-tuned model
model_save_path = '/content/fine_tuned_model_with_triplets'
model.save(model_save_path)
fine_tuned_model = SentenceTransformer(model_save_path)
